# Analyse d'évolution des prix CCAM par menu (VERSION FINALE)

**Approche optimisée** : filtrage avant jointure + traitement menu par menu

Critères :
- **Grilles** : 3, 4, 5, 7, 17, 18
- **Activités** : 1, 2, 3, 4  
- **Phase** : 0

## 1. Imports et Configuration

In [1]:
from pathlib import Path
import pandas as pd
import pickle
from datetime import datetime

# Configuration
DATA_DIR = Path('data/pickle')
OUTPUT_DIR = Path('output_by_menu')
OUTPUT_DIR.mkdir(exist_ok=True)

# Critères de filtrage
GRILLES_FILTER = [3, 4, 5, 7, 17, 18]
ACTIVITES_FILTER = [1, 2, 3, 4]
PHASE_FILTER = 0

print("✓ Configuration chargée")
print(f"  Grilles : {GRILLES_FILTER}")
print(f"  Activités : {ACTIVITES_FILTER}")
print(f"  Phase : {PHASE_FILTER}")

✓ Configuration chargée
  Grilles : [3, 4, 5, 7, 17, 18]
  Activités : [1, 2, 3, 4]
  Phase : 0


## 2. Chargement des données (pickle)

In [2]:
def load_pickle(filename):
    """Charge un fichier pickle"""
    with open(DATA_DIR / filename, 'rb') as f:
        return pickle.load(f)

print("📊 Chargement des tables...\n")

df_acte = load_pickle('R_ACTE.pkl')
print(f"✓ R_ACTE : {len(df_acte):,} lignes")

df_acte_ivite = load_pickle('R_ACTE_IVITE.pkl')
print(f"✓ R_ACTE_IVITE : {len(df_acte_ivite):,} lignes")

df_acte_ivite_phase = load_pickle('R_ACTE_IVITE_PHASE.pkl')
print(f"✓ R_ACTE_IVITE_PHASE : {len(df_acte_ivite_phase):,} lignes")

df_pu_base = load_pickle('R_PU_BASE.pkl')
print(f"✓ R_PU_BASE : {len(df_pu_base):,} lignes")

df_menu = load_pickle('R_MENU.pkl')
print(f"✓ R_MENU : {len(df_menu):,} lignes")

df_activite = load_pickle('R_ACTIVITE.pkl')
print(f"✓ R_ACTIVITE : {len(df_activite):,} lignes")

df_grille = load_pickle('R_TB23.pkl')
print(f"✓ R_TB23 (grilles) : {len(df_grille):,} lignes")

print("\n✅ Toutes les tables chargées")

📊 Chargement des tables...

✓ R_ACTE : 82,890 lignes
✓ R_ACTE_IVITE : 137,481 lignes
✓ R_ACTE_IVITE_PHASE : 137,742 lignes
✓ R_PU_BASE : 772,066 lignes
✓ R_MENU : 1,725 lignes
✓ R_ACTIVITE : 5 lignes
✓ R_TB23 (grilles) : 20 lignes

✅ Toutes les tables chargées


## 3. Conversion des types de données (FIX)

In [3]:
print("🔧 CONVERSION DES TYPES DE DONNÉES\n")

# Convertir activ_cod en entier
print("Conversion de activ_cod en entier...")
print(f"  Avant : type={df_acte_ivite['activ_cod'].dtype}, valeurs={df_acte_ivite['activ_cod'].unique()[:5]}")
df_acte_ivite['activ_cod'] = pd.to_numeric(df_acte_ivite['activ_cod'], errors='coerce').astype('Int64')
print(f"  Après : type={df_acte_ivite['activ_cod'].dtype}, valeurs={sorted(df_acte_ivite['activ_cod'].unique())}")

# Convertir grille_cod en entier
print("\nConversion de grille_cod en entier...")
print(f"  Avant : type={df_pu_base['grille_cod'].dtype}")
df_pu_base['grille_cod'] = pd.to_numeric(df_pu_base['grille_cod'], errors='coerce').astype('Int64')
print(f"  Après : type={df_pu_base['grille_cod'].dtype}")

# Convertir phase_cod en entier
print("\nConversion de phase_cod en entier...")
print(f"  Avant : type={df_acte_ivite_phase['phase_cod'].dtype}")
df_acte_ivite_phase['phase_cod'] = pd.to_numeric(df_acte_ivite_phase['phase_cod'], errors='coerce').astype('Int64')
print(f"  Après : type={df_acte_ivite_phase['phase_cod'].dtype}")

print("\n✅ Conversions terminées")

🔧 CONVERSION DES TYPES DE DONNÉES

Conversion de activ_cod en entier...
  Avant : type=object, valeurs=['1' '4' '5' '2' '3']
  Après : type=Int64, valeurs=[1, 2, 3, 4, 5]

Conversion de grille_cod en entier...
  Avant : type=int64
  Après : type=Int64

Conversion de phase_cod en entier...
  Avant : type=int64
  Après : type=Int64

✅ Conversions terminées


## 4. PRÉ-FILTRAGE (clé de l'optimisation !)

In [4]:
print("🔍 PRÉ-FILTRAGE DES TABLES (avant jointure)\n")

# 1. Filtrer R_PU_BASE sur les grilles
print(f"1. R_PU_BASE : {len(df_pu_base):,} lignes")
df_pu_base_filtered = df_pu_base[df_pu_base['grille_cod'].isin(GRILLES_FILTER)].copy()
print(f"   → après filtre grilles : {len(df_pu_base_filtered):,} lignes ({len(df_pu_base_filtered)/len(df_pu_base)*100:.1f}%)")

# 2. Filtrer R_ACTE_IVITE_PHASE sur la phase
print(f"\n2. R_ACTE_IVITE_PHASE : {len(df_acte_ivite_phase):,} lignes")
df_acte_ivite_phase_filtered = df_acte_ivite_phase[df_acte_ivite_phase['phase_cod'] == PHASE_FILTER].copy()
print(f"   → après filtre phase : {len(df_acte_ivite_phase_filtered):,} lignes ({len(df_acte_ivite_phase_filtered)/len(df_acte_ivite_phase)*100:.1f}%)")

# 3. Filtrer R_ACTE_IVITE sur les activités
print(f"\n3. R_ACTE_IVITE : {len(df_acte_ivite):,} lignes")
df_acte_ivite_filtered = df_acte_ivite[df_acte_ivite['activ_cod'].isin(ACTIVITES_FILTER)].copy()
print(f"   → après filtre activités : {len(df_acte_ivite_filtered):,} lignes ({len(df_acte_ivite_filtered)/len(df_acte_ivite)*100:.1f}%)")

print("\n✅ Pré-filtrage terminé - tables réduites !")

🔍 PRÉ-FILTRAGE DES TABLES (avant jointure)

1. R_PU_BASE : 772,066 lignes
   → après filtre grilles : 193,525 lignes (25.1%)

2. R_ACTE_IVITE_PHASE : 137,742 lignes
   → après filtre phase : 137,245 lignes (99.6%)

3. R_ACTE_IVITE : 137,481 lignes
   → après filtre activités : 135,506 lignes (98.6%)

✅ Pré-filtrage terminé - tables réduites !


## 5. Identifier les codes AAP valides (intersection)

In [5]:
print("🔗 Identification des codes AAP valides (qui respectent tous les critères)\n")

# Codes AAP qui ont des prix dans les bonnes grilles
aap_codes_with_prices = set(df_pu_base_filtered['aap_cod'].unique())
print(f"Codes AAP avec prix dans grilles {GRILLES_FILTER} : {len(aap_codes_with_prices):,}")

# Codes AAP avec phase 0
aap_codes_with_phase = set(df_acte_ivite_phase_filtered['cod_aap'].unique())
print(f"Codes AAP avec phase {PHASE_FILTER} : {len(aap_codes_with_phase):,}")

# Intersection : codes AAP valides
valid_aap_codes = aap_codes_with_prices & aap_codes_with_phase
print(f"\n✓ Codes AAP valides (intersection) : {len(valid_aap_codes):,}")

# Filtrer les tables sur les codes AAP valides
df_pu_base_valid = df_pu_base_filtered[df_pu_base_filtered['aap_cod'].isin(valid_aap_codes)].copy()
df_acte_ivite_phase_valid = df_acte_ivite_phase_filtered[df_acte_ivite_phase_filtered['cod_aap'].isin(valid_aap_codes)].copy()

print(f"\nR_PU_BASE final : {len(df_pu_base_valid):,} lignes")
print(f"R_ACTE_IVITE_PHASE final : {len(df_acte_ivite_phase_valid):,} lignes")

🔗 Identification des codes AAP valides (qui respectent tous les critères)

Codes AAP avec prix dans grilles [3, 4, 5, 7, 17, 18] : 13,466
Codes AAP avec phase 0 : 13,697

✓ Codes AAP valides (intersection) : 13,407

R_PU_BASE final : 192,782 lignes
R_ACTE_IVITE_PHASE final : 136,213 lignes


## 6. Créer des dictionnaires de lookup (évite les gros merge)

In [6]:
print("📚 Création des dictionnaires de lookup...\n")

# Dictionnaire menu_cod -> info menu
menu_dict = df_menu.set_index('cod_menu')[['libelle', 'cod_pere']].to_dict('index')
print(f"✓ menu_dict : {len(menu_dict):,} entrées")

# Dictionnaire cod_acte -> info acte (GESTION DES DOUBLONS)
print(f"   R_ACTE brut : {len(df_acte):,} lignes")
print(f"   Codes actes uniques : {df_acte['cod_acte'].nunique():,}")

# Convertir dt_modif en datetime si nécessaire
if not pd.api.types.is_datetime64_any_dtype(df_acte['dt_modif']):
    df_acte['dt_modif'] = pd.to_datetime(df_acte['dt_modif'], errors='coerce')

# Trier par date et garder la plus récente pour chaque cod_acte
df_acte_unique = df_acte.sort_values('dt_modif', ascending=False).drop_duplicates(subset='cod_acte', keep='first')
print(f"   Après dédoublonnage : {len(df_acte_unique):,} lignes")

acte_dict = df_acte_unique.set_index('cod_acte')[['nom_court', 'nom_long', 'menu_cod']].to_dict('index')
print(f"✓ acte_dict : {len(acte_dict):,} entrées")

# Dictionnaire cod_activ -> libellé
activite_dict = df_activite.set_index('cod_activ')['libelle'].to_dict()
print(f"✓ activite_dict : {len(activite_dict):,} entrées")

# Dictionnaire cod_grille -> libellé
grille_dict = df_grille.set_index('cod_grille')['libelle'].to_dict()
print(f"✓ grille_dict : {len(grille_dict):,} entrées")

# Dictionnaire cod_aa -> info (GESTION DES DOUBLONS)
print(f"   R_ACTE_IVITE filtré : {len(df_acte_ivite_filtered):,} lignes")
print(f"   Codes AA uniques : {df_acte_ivite_filtered['cod_aa'].nunique():,}")

# Supprimer les doublons en gardant le premier
df_acte_ivite_unique = df_acte_ivite_filtered.drop_duplicates(subset='cod_aa', keep='first')
print(f"   Après dédoublonnage : {len(df_acte_ivite_unique):,} lignes")

aa_dict = df_acte_ivite_unique.set_index('cod_aa')[['acte_cod', 'activ_cod']].to_dict('index')
print(f"✓ aa_dict : {len(aa_dict):,} entrées")

print("\n✅ Dictionnaires créés - accès rapide !")

📚 Création des dictionnaires de lookup...

✓ menu_dict : 1,725 entrées
   R_ACTE brut : 82,890 lignes
   Codes actes uniques : 8,546
   Après dédoublonnage : 8,546 lignes
✓ acte_dict : 8,546 entrées
✓ activite_dict : 5 entrées
✓ grille_dict : 20 entrées
   R_ACTE_IVITE filtré : 135,506 lignes
   Codes AA uniques : 13,547
   Après dédoublonnage : 13,547 lignes
✓ aa_dict : 13,547 entrées

✅ Dictionnaires créés - accès rapide !


## 7. Regrouper les données par menu (sans jointure lourde)

In [7]:
print("📋 Regroupement par MENUS DE 1ER NIVEAU (cod_pere = 1)...\n")

from collections import defaultdict

def get_top_level_menu(menu_cod, menu_dict, max_depth=20):
    """Remonte la hiérarchie jusqu'au menu de 1er niveau (cod_pere = 1)"""
    current_cod = menu_cod
    depth = 0

    while depth < max_depth:
        if pd.isna(current_cod) or current_cod is None or current_cod == 0:
            return None

        menu_info = menu_dict.get(current_cod)
        if menu_info is None:
            return None

        cod_pere = menu_info.get('cod_pere')

        # Si le parent est 1 (racine), on a trouvé le menu de 1er niveau
        if cod_pere == 1:
            return current_cod

        # Si le parent est 0 ou None, on est à la racine
        if pd.isna(cod_pere) or cod_pere == 0:
            return None

        # Remonter au parent
        current_cod = cod_pere
        depth += 1

    return None

# Regroupement par menu de 1er niveau
menu_to_aap = defaultdict(list)

print(f"Codes AAP valides à traiter : {len(valid_aap_codes):,}")

processed = 0
skipped_no_aa = 0
skipped_no_acte = 0
skipped_no_menu = 0
skipped_not_level1 = 0

for aap_code in valid_aap_codes:
    # Trouver le code AA
    aap_rows = df_acte_ivite_phase_valid[df_acte_ivite_phase_valid['cod_aap'] == aap_code]
    if aap_rows.empty:
        skipped_no_aa += 1
        continue

    aa_code = aap_rows.iloc[0]['aa_cod']

    # Lookup du code acte
    if aa_code not in aa_dict:
        skipped_no_aa += 1
        continue

    acte_code = aa_dict[aa_code]['acte_cod']

    # Lookup du menu
    if acte_code not in acte_dict:
        skipped_no_acte += 1
        continue

    menu_cod = acte_dict[acte_code]['menu_cod']

    # Vérifier que menu_cod n'est pas NaN ou None
    if pd.isna(menu_cod) or menu_cod is None or menu_cod == 0:
        skipped_no_menu += 1
        continue

    # Remonter au menu de 1er niveau
    top_level_menu = get_top_level_menu(menu_cod, menu_dict)

    if top_level_menu is None:
        skipped_not_level1 += 1
        continue

    menu_to_aap[top_level_menu].append(aap_code)
    processed += 1

print(f"\n✓ Traitement terminé :")
print(f"   Codes AAP traités : {processed:,}")
print(f"   Sans code AA : {skipped_no_aa:,}")
print(f"   Sans code acte : {skipped_no_acte:,}")
print(f"   Sans menu : {skipped_no_menu:,}")
print(f"   Sans menu de 1er niveau : {skipped_not_level1:,}")
print(f"\n✓ {len(menu_to_aap)} menus de 1er niveau identifiés")

if len(menu_to_aap) > 0:
    print(f"\nMenus de 1er niveau (cod_pere = 1) :")
    sorted_menus = sorted(menu_to_aap.items(), key=lambda x: len(x[1]), reverse=True)
    for menu_cod, aap_list in sorted_menus:
        menu_name = menu_dict.get(menu_cod, {}).get('libelle', 'Inconnu')
        print(f"  Menu {menu_cod:4d} : {len(aap_list):4d} AAP - {menu_name[:60]}")
else:
    print("\n❌ Aucun menu de 1er niveau trouvé !")

📋 Regroupement par MENUS DE 1ER NIVEAU (cod_pere = 1)...

Codes AAP valides à traiter : 13,407

✓ Traitement terminé :
   Codes AAP traités : 13,229
   Sans code AA : 178
   Sans code acte : 0
   Sans menu : 0
   Sans menu de 1er niveau : 0

✓ 19 menus de 1er niveau identifiés

Menus de 1er niveau (cod_pere = 1) :
  Menu  648 : 2161 AAP - APPAREIL DIGESTIF
  Menu  249 : 2012 AAP - APPAREIL CIRCULATOIRE
  Menu  858 : 1518 AAP - APPAREIL URINAIRE ET GÉNITAL
  Menu 1304 : 1038 AAP - APPAREIL OSTÉOARTICULAIRE ET MUSCULAIRE DU MEMBRE INFÉRIEUR
  Menu 1174 :  867 AAP - APPAREIL OSTÉOARTICULAIRE ET MUSCULAIRE DU MEMBRE SUPÉRIEUR
  Menu    2 :  824 AAP - SYSTÈME NERVEUX CENTRAL, PÉRIPHÉRIQUE ET AUTONOME
  Menu  569 :  729 AAP - APPAREIL RESPIRATOIRE
  Menu 1456 :  702 AAP - SYSTÈME TÉGUMENTAIRE - GLANDE MAMMAIRE
  Menu  124 :  630 AAP - OEIL ET ANNEXES
  Menu 1108 :  599 AAP - APPAREIL OSTÉOARTICULAIRE ET MUSCULAIRE DU COU ET DU TRONC
  Menu 1053 :  576 AAP - APPAREIL OSTÉOARTICULAIRE ET MUSCU

## 8. Fonction d'analyse optimisée par menu

In [ ]:
def analyze_menu_optimized(menu_cod, aap_codes_list):
    """Analyse optimisée pour un menu donné (pas de gros merge !)"""
    
    # Filtrer uniquement les AAP de ce menu
    df_menu_pu = df_pu_base_valid[df_pu_base_valid['aap_cod'].isin(aap_codes_list)].copy()
    
    if df_menu_pu.empty:
        return None
    
    # Convertir dates
    if not pd.api.types.is_datetime64_any_dtype(df_menu_pu['apdt_modif']):
        df_menu_pu['apdt_modif'] = pd.to_datetime(df_menu_pu['apdt_modif'], errors='coerce')
    
    results = []
    
    # Pour chaque code AAP
    for aap_code in df_menu_pu['aap_cod'].unique():
        df_aap = df_menu_pu[df_menu_pu['aap_cod'] == aap_code]
        
        # Trouver les infos via dictionnaires (rapide !)
        aap_row = df_acte_ivite_phase_valid[df_acte_ivite_phase_valid['cod_aap'] == aap_code].iloc[0]
        aa_code = aap_row['aa_cod']
        
        if aa_code not in aa_dict:
            continue
        
        acte_code = aa_dict[aa_code]['acte_cod']
        activ_cod = aa_dict[aa_code]['activ_cod']
        
        if acte_code not in acte_dict:
            continue
        
        acte_info = acte_dict[acte_code]
        
        # Pour chaque grille
        for grille in GRILLES_FILTER:
            df_grille_data = df_aap[df_aap['grille_cod'] == grille].sort_values('apdt_modif')
            
            if not df_grille_data.empty:
                first = df_grille_data.iloc[0]
                last = df_grille_data.iloc[-1]
                
                result = {
                    'cod_acte': acte_code,
                    'nom_court': acte_info.get('nom_court', ''),
                    'nom_long': acte_info.get('nom_long', ''),
                    'activite': activ_cod,
                    'activite_libelle': activite_dict.get(activ_cod, ''),
                    'grille_cod': grille,
                    'grille_libelle': grille_dict.get(grille, f'Grille {grille}'),
                    'date_premiere_modif': first['apdt_modif'],
                    'prix_initial': first['pu_base'],
                    'date_derniere_modif': last['apdt_modif'],
                    'prix_actuel': last['pu_base'],
                    'evolution_euros': last['pu_base'] - first['pu_base'],
                    'evolution_%': round(((last['pu_base'] - first['pu_base']) / first['pu_base'] * 100) if first['pu_base'] != 0 else 0,2),
                    'nb_modifications': len(df_grille_data)
                }
                results.append(result)
    
    return pd.DataFrame(results) if results else None

print("✓ Fonction d'analyse optimisée définie")

✓ Fonction d'analyse optimisée définie


## 9. Fonction d'export Excel

In [9]:
def export_menu_to_excel(menu_cod, df_analysis):
    """Exporte l'analyse d'un menu en Excel"""
    if df_analysis is None or df_analysis.empty:
        return None
    
    # Nom du menu
    menu_info = menu_dict.get(menu_cod, {})
    menu_name = menu_info.get('libelle', f'Menu_{menu_cod}')
    menu_name = "".join(c if c.isalnum() or c in (' ', '-', '_') else '_' for c in menu_name)
    menu_name = menu_name[:100]
    
    filename = OUTPUT_DIR / f"{menu_cod}_{menu_name}.xlsx"
    
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        # Feuille 1
        df_analysis.to_excel(writer, sheet_name='Evolution_prix', index=False)
        
        # Feuille 2
        if len(df_analysis) > 0:
            stats_by_grille = df_analysis.groupby(['grille_cod', 'grille_libelle']).agg({
                'cod_acte': 'count',
                'prix_initial': 'mean',
                'prix_actuel': 'mean',
                'evolution_euros': 'mean',
                'evolution_pct': 'mean'
            }).round(2)
            stats_by_grille.columns = ['Nb_actes', 'Prix_initial_moyen', 'Prix_actuel_moyen', 'Evolution_€_moyenne', 'Evolution_%_moyenne']
            stats_by_grille.to_excel(writer, sheet_name='Stats_par_grille')
        
        # Feuille 3
        if len(df_analysis) > 0:
            top_n = min(20, len(df_analysis))
            top_evolutions = df_analysis.nlargest(top_n, 'evolution_euros')[[
                'cod_acte', 'nom_court', 'grille_cod', 'prix_initial', 'prix_actuel', 'evolution_euros', 'evolution_pct'
            ]]
            top_evolutions.to_excel(writer, sheet_name='Top_evolutions', index=False)
    
    return filename

print("✓ Fonction d'export définie")

✓ Fonction d'export définie


## 10. Test sur un menu

In [10]:
if len(menu_to_aap) > 0:
    # Test sur le premier menu
    test_menu = list(menu_to_aap.keys())[0]
    test_aap_list = menu_to_aap[test_menu]

    print(f"🧪 Test sur menu {test_menu}")
    print(f"   {len(test_aap_list)} codes AAP")

    df_test = analyze_menu_optimized(test_menu, test_aap_list)

    if df_test is not None:
        print(f"\n✓ Analyse terminée : {len(df_test)} lignes")
        print(f"\nAperçu :")
        display(df_test.head(10))

        # Export
        test_file = export_menu_to_excel(test_menu, df_test)
        print(f"\n✓ Excel créé : {test_file.name}")
    else:
        print("❌ Aucune donnée pour ce menu")
else:
    print("❌ Aucun menu disponible pour le test")

🧪 Test sur menu 124
   630 codes AAP

✓ Analyse terminée : 3774 lignes

Aperçu :


,cod_acte,nom_court,nom_long,activite,activite_libelle,grille_cod,grille_libelle,date_premiere_modif,prix_initial,date_derniere_modif,prix_actuel,evolution_euros,evolution_pct,nb_modifications
0,BEPB001,goniotomie oculaire v.transclérale,"Goniotomie oculaire, par voie transsclérale",4,,3,"Spé chir et gynéco-obst, s1 / s1 OPTAM / s1 O...",2017-10-16,79.54,2024-11-04,80.86,1.32,1.659542,4
1,BEPB001,goniotomie oculaire v.transclérale,"Goniotomie oculaire, par voie transsclérale",4,,4,"Spé chir et gynéco-obst, s2-1DP",2017-10-16,79.54,2024-11-04,80.86,1.32,1.659542,4
2,BEPB001,goniotomie oculaire v.transclérale,"Goniotomie oculaire, par voie transsclérale",4,,5,"Spé chir et gynéco-obst, s2-1DP OPTAM",2017-10-16,79.54,2024-11-04,80.86,1.32,1.659542,4
3,BEPB001,goniotomie oculaire v.transclérale,"Goniotomie oculaire, par voie transsclérale",4,,7,"Anesthésistes, s1 / s1-1DP-2 OPTAM",2017-10-16,79.54,2022-03-03,79.54,0.00,0.000000,3
4,BEPB001,goniotomie oculaire v.transclérale,"Goniotomie oculaire, par voie transsclérale",4,,17,Anesthésistes OPTAM/OPTAMACO (s1/s2/s1DP),2024-11-04,80.86,2024-11-04,80.86,0.00,0.000000,1
5,BEPB001,goniotomie oculaire v.transclérale,"Goniotomie oculaire, par voie transsclérale",4,,18,Anesthésistes s1,2024-11-04,80.86,2024-11-04,80.86,0.00,0.000000,1
6,BAMA013,répar. paup. inf. lambeau rég. +gref.,Réparation de perte de substance de la paupièr...,4,,3,"Spé chir et gynéco-obst, s1 / s1 OPTAM / s1 O...",2017-10-16,92.01,2024-11-04,93.62,1.61,1.749810,4
7,BAMA013,répar. paup. inf. lambeau rég. +gref.,Réparation de perte de substance de la paupièr...,4,,4,"Spé chir et gynéco-obst, s2-1DP",2017-10-16,92.01,2024-11-04,93.62,1.61,1.749810,4
8,BAMA013,répar. paup. inf. lambeau rég. +gref.,Réparation de perte de substance de la paupièr...,4,,5,"Spé chir et gynéco-obst, s2-1DP OPTAM",2017-10-16,92.01,2024-11-04,93.62,1.61,1.749810,4
9,BAMA013,répar. paup. inf. lambeau rég. +gref.,Réparation de perte de substance de la paupièr...,4,,7,"Anesthésistes, s1 / s1-1DP-2 OPTAM",2017-10-16,92.01,2022-03-03,92.01,0.00,0.000000,3



✓ Excel créé : 124_OEIL ET ANNEXES.xlsx


## 11. Génération pour TOUS les menus

⚠️ Beaucoup plus rapide maintenant grâce à l'optimisation !

In [11]:
if len(menu_to_aap) > 0:
    print("="*80)
    print("GÉNÉRATION POUR TOUS LES MENUS (OPTIMISÉ)")
    print("="*80)

    total = len(menu_to_aap)
    generated = 0
    skipped = 0

    for i, (menu_cod, aap_list) in enumerate(sorted(menu_to_aap.items()), 1):

        if i % 10 == 0:
            print(f"\n[{i}/{total}] Progression : {i*100//total}%")

        df_analysis = analyze_menu_optimized(menu_cod, aap_list)

        if df_analysis is not None and not df_analysis.empty:
            filename = export_menu_to_excel(menu_cod, df_analysis)
            generated += 1
            if i <= 5 or i % 50 == 0:
                print(f"  Menu {menu_cod:4d} : {len(df_analysis):3d} analyses → {filename.name[:50]}")
        else:
            skipped += 1

    print("\n" + "="*80)
    print(f"✅ TERMINÉ")
    print(f"   Fichiers générés : {generated}")
    print(f"   Menus sans données : {skipped}")
    print(f"   Dossier : {OUTPUT_DIR}")
    print("="*80)
else:
    print("❌ Aucun menu disponible - vérifier les filtres")

GÉNÉRATION POUR TOUS LES MENUS (OPTIMISÉ)
  Menu    2 : 4942 analyses → 2_SYSTÈME NERVEUX CENTRAL_ PÉRIPHÉRIQUE ET AUTONOM
  Menu  124 : 3774 analyses → 124_OEIL ET ANNEXES.xlsx
  Menu  211 : 1337 analyses → 211_OREILLE.xlsx
  Menu  249 : 12061 analyses → 249_APPAREIL CIRCULATOIRE.xlsx
  Menu  533 : 1338 analyses → 533_SYSTÈME IMMUNITAIRE ET SYSTÈME HÉMATOPOÏÉTIQUE

[10/19] Progression : 52%

✅ TERMINÉ
   Fichiers générés : 19
   Menus sans données : 0
   Dossier : output_by_menu


## 12. Résumé

In [12]:
excel_files = list(OUTPUT_DIR.glob("*.xlsx"))
total_size = sum(f.stat().st_size for f in excel_files)

print(f"📊 Résumé final")
print(f"   Fichiers Excel : {len(excel_files)}")
print(f"   Taille totale : {total_size / (1024*1024):.1f} MB")
print(f"   Dossier : {OUTPUT_DIR.absolute()}")

if len(excel_files) > 0:
    print(f"\n📈 Top 5 des plus gros fichiers :")
    file_sizes = [(f, f.stat().st_size) for f in excel_files]
    file_sizes.sort(key=lambda x: x[1], reverse=True)
    for i, (f, size) in enumerate(file_sizes[:5], 1):
        print(f"   {i}. {f.name[:70]} ({size/1024:.1f} KB)")

📊 Résumé final
   Fichiers Excel : 19
   Taille totale : 5.6 MB
   Dossier : /Users/mrouer/python_temp/ccam_analysis/output_by_menu

📈 Top 5 des plus gros fichiers :
   1. 648_APPAREIL DIGESTIF.xlsx (925.2 KB)
   2. 249_APPAREIL CIRCULATOIRE.xlsx (867.1 KB)
   3. 858_APPAREIL URINAIRE ET GÉNITAL.xlsx (639.1 KB)
   4. 1304_APPAREIL OSTÉOARTICULAIRE ET MUSCULAIRE DU MEMBRE INFÉRIEUR.xlsx (450.0 KB)
   5. 1174_APPAREIL OSTÉOARTICULAIRE ET MUSCULAIRE DU MEMBRE SUPÉRIEUR.xlsx (369.1 KB)
